In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import serviceHandlers as handlers
from serviceHandlers import NamSorEndpoint


In [2]:
# Getting a dataset

df = pd.read_csv("../../data/all.csv")
df = df[df['hasMiddleName']==False].iloc[0:10]
df

,Unnamed: 0,fullName,firstName,lastName,gender,isoCountry,continent,birthYear,hasMiddleName,hasNoLastName,source
0,0,Clara Benson,Clara,Benson,female,GH,Africa,2000,False,False,wikidata
2,2,Dalia Ziada,Dalia,Ziada,female,EG,Africa,1982,False,False,wikidata
3,3,Fatou Haidara,Fatou,Haidara,female,ML,Africa,1962,False,False,wikidata
4,4,Claude Haffner,Claude,Haffner,female,NaN,NaN,1976,False,False,wikidata
7,7,Diallo Sène,Diallo,Sène,female,ML,Africa,1952,False,False,wikidata
8,8,Fatoumata Sylla,Fatoumata,Sylla,female,ML,Africa,1954,False,False,wikidata
11,11,Manuella Ollo,Manuella,Ollo,female,NaN,NaN,1985,False,False,wikidata
12,12,Awa Traoré,Awa,Traoré,female,ML,Africa,1980,False,False,wikidata
17,17,Lorna Doorman,Lorna,Doorman,female,ZW,Africa,1998,False,False,wikidata
18,18,Enayat al-Zayyat,Enayat,al-Zayyat,female,EG,Africa,1936,False,False,wikidata


# Testing synchronous wrapper

## Testing GenderizeIoHandler

In [3]:
# genderizeIO_handler = handlers.GenderizeIoHandler(datasource=df, hasSubscription=False)
# genderizeIO_handler.get_prediction(True)

## Testing GenderAPI

In [4]:
# genderAPI_IO_Handler = handlers.GenderAPI_IO_Handler(datasource=df)

# genderAPI_IO_Handler.callAPI(useLocalization=True)

## Testing GenderGuesser

In [5]:
# genderGuesserHandler = handlers.GenderGuesserHandler(df)

# genderGuesserHandler.get_prediction(False)

## Testing GenderAPI.com 

In [6]:
# genderAPI_com_handler = handlers.GenderAPI_com_Handler(df)

# genderAPI_com_handler.get_prediction(True, True)

## Testing NamSor Handler

In [7]:
# namSor_handler = handlers.NamSorHandler(df)

# namSor_handler.get_prediction(endpoint=NamSorEndpoint.FULL_NAME)

## Testing NameAPI

In [8]:
# nameAPI_handler = handlers.NameAPIHandler(df)

# await nameAPI_handler.get_prediction(True)

# Testing async Wrapper

In [22]:
import serviceWrapperAsync as wrapper
from serviceWrapperAsync import NamSorEndpoint as endpoint

In [23]:
# genderize_io_wrapper = wrapper.GenderizeWrapper(df)
# nameAPI_wrapper = wrapper.NameAPIWrapper(df)
namSor_wrapper = wrapper.NamSorWrapper(df)
# genderCom_wrapper = wrapper.GenderAPI_com_Wrapper(df)
# genderIO_wrapper = wrapper.GenderAPI_IO_Wrapper(df)

In [25]:
result = await namSor_wrapper.get_prediction_async(endpoint=endpoint.FIRST_NAME_GEO)
result

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraGenderScale,extraScore,extraProbabilityCalibrated,extraScript
0,0,Clara Benson,Clara,female,female,GH,True,NamSor,0.999099,15.669216,0.999549,LATIN
1,1,Dalia Ziada,Dalia,female,female,EG,True,NamSor,0.998789,15.373957,0.999395,LATIN
2,2,Fatou Haidara,Fatou,female,female,ML,True,NamSor,0.987619,12.789984,0.993810,LATIN
3,3,Claude Haffner,Claude,female,male,NaN,True,NamSor,-0.995861,14.144691,0.997930,LATIN
4,4,Diallo Sène,Diallo,female,female,ML,True,NamSor,0.310045,1.876022,0.655023,LATIN
5,5,Fatoumata Sylla,Fatoumata,female,female,ML,True,NamSor,0.999991,20.272138,0.999995,LATIN
6,6,Manuella Ollo,Manuella,female,female,NaN,True,NamSor,0.999996,20.983707,0.999998,LATIN
7,7,Awa Traoré,Awa,female,female,ML,True,NamSor,0.999990,20.152528,0.999995,LATIN
8,8,Lorna Doorman,Lorna,female,female,ZW,True,NamSor,0.999271,15.880959,0.999635,LATIN
9,9,Enayat al-Zayyat,Enayat,female,male,EG,True,NamSor,-0.971284,10.579773,0.985642,LATIN


# testing wrapper manager

In [27]:
%load_ext autoreload
%autoreload 2

import wrapperManager as wrapman


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
manager = wrapman.WrapperManager(df)
results = await manager.run_subset(
            ["genderize", "namsor"],
            config={"genderize": {"useLocalization": True}, "namsor": {"endpoint": endpoint.FIRST_NAME_GEO}}
        )

In [29]:
resultGenderize = results['genderize']
resultNamSor = results['namsor']

sameCols = [col for col in resultNamSor.columns if col in resultGenderize.columns]
print(sameCols)

['index', 'fullName', 'namePassed', 'correctGender', 'predictedGender', 'localization', 'useLocalization', 'serviceUsed']


In [30]:
testConcat = pd.concat([resultGenderize[sameCols], resultNamSor[sameCols]])
testConcat

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed
0,0,Clara Benson,Clara,female,female,GH,True,genderize.IO
1,1,Dalia Ziada,Dalia,female,female,EG,True,genderize.IO
2,2,Fatou Haidara,Fatou,female,female,ML,True,genderize.IO
3,3,Claude Haffner,Claude,female,male,NaN,True,genderize.IO
4,4,Diallo Sène,Diallo,female,male,ML,True,genderize.IO
5,5,Fatoumata Sylla,Fatoumata,female,female,ML,True,genderize.IO
6,6,Manuella Ollo,Manuella,female,female,NaN,True,genderize.IO
7,7,Awa Traoré,Awa,female,female,ML,True,genderize.IO
8,8,Lorna Doorman,Lorna,female,female,ZW,True,genderize.IO
9,9,Enayat al-Zayyat,Enayat,female,male,EG,True,genderize.IO
